In [1]:
import Pkg; Pkg.activate("C:/Users/s151781/AppData/Local/Julia-1.3.1/GN/Project.toml")
using Revise
using Plots
using FFTW
using Compat
using WAV
using DSP
using Base64
using ForneyLab
using LinearAlgebra
using ProgressMeter

Activating environment at `C:\Users\s151781\AppData\Local\Julia-1.3.1\GN\Project.toml`


┌ Info: Precompiling Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1273


In [27]:
include("../extensions/complex_gaussian.jl")
include("../extensions/hgf.jl")
include("../functions/auxiliary/workflow.jl")


em (generic function with 1 method)

In [19]:
# generate data
import Distributions: Normal, MvNormal, MixtureModel, Dirichlet
N = 1
nr_freqs = 2
n_samples = N

μ_ξi = 1.0*collect(-4:2:4)
Σ_ξ = 1e-4*diagm(ones(nr_freqs))
Σ_meas = 1e-10*diagm(ones(nr_freqs))

ξ_samples = Array{Float64,2}(undef, N, nr_freqs)
X_samples = Array{Complex{Float64},2}(undef, N, nr_freqs)
y_samples = Array{Complex{Float64},2}(undef, N, nr_freqs)

for n = 1:N

#     sample_ξ = rand(MvNormal(μ_ξ, Σ_ξ))
#     sample_X = rand(MvNormal(zeros(nr_freqs), real.((exp.(0.5*sample_ξ))))) + 1im*rand(MvNormal(zeros(nr_freqs), real.((exp.(0.5*sample_ξ)))))
#     sample_y = rand(MvNormal(real(sample_X), 0.5*real.(Σ_meas))) + 1im*rand(MvNormal(imag(sample_X), 0.5*real.(Σ_meas)))
    
#     ξ_samples[n,:] = sample_ξ
#     X_samples[n,:] = sample_X
#     y_samples[n,:] = sample_y
    for k = 1:nr_freqs
        sample_ξ = rand(Normal(μ_ξi[k], sqrt(Σ_ξ[k,k])))
        sample_X = rand(Normal(0, sqrt(0.5*exp(sample_ξ)))) + 1im*rand(Normal(0, sqrt(0.5*exp(sample_ξ))))
        sample_y = rand(Normal(real(sample_X), sqrt(0.5*Σ_meas[k,k]))) + 1im*rand(Normal(imag(sample_X), sqrt(0.5*Σ_meas[k,k])))

        ξ_samples[n,k] = sample_ξ
        X_samples[n,k] = sample_X
        y_samples[n,k] = sample_y
    end
end

t = collect(1:N)
Y = y_samples

1×2 Array{Complex{Float64},2}:
 0.0973786-0.0844656im  0.0229807-0.490655im

# Building graph

In [42]:
#nr_freqs = dimension

N_clusters = 3

fg = FactorGraph()
α = 0.5
# Specify generative model
@RV _pi ~ ForneyLab.Dirichlet(α*ones(N_clusters))
@RV m_1 ~ GaussianMeanVariance(zeros(nr_freqs), 100*diagm(ones(nr_freqs)))
@RV w_1 ~ Wishart(diagm(ones(nr_freqs)), 5.0)
@RV m_2 ~ GaussianMeanVariance(zeros(nr_freqs), 100*diagm(ones(nr_freqs)))
@RV w_2 ~ Wishart(diagm(ones(nr_freqs)), 5.0)
@RV m_3 ~ GaussianMeanVariance(zeros(nr_freqs), 100*diagm(ones(nr_freqs)))
@RV w_3 ~ Wishart(diagm(ones(nr_freqs)), 5.0)

z = Vector{Variable}(undef, n_samples)
ξ = Vector{Variable}(undef, n_samples)
X = Vector{Variable}(undef, n_samples)
y = Vector{Variable}(undef, n_samples)
for i in 1:n_samples
    @RV z[i] ~ Categorical(_pi)
    @RV ξ[i] ~ GaussianMixture(z[i], m_1, w_1, m_2, w_2, m_3, w_3)
    # HGF
    @RV X[i] ~ HGF(ξ[i])

    # observation model
    @RV y[i] ~ ComplexNormal(X[i], 1e-10*diagm(ones(nr_freqs)).+0im, mat(0.0+0.0im))
    
    
    placeholder(y[i], :y, index=i, dims=(nr_freqs,))
end
# draw graph
ForneyLab.draw(fg)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 2.38.0 (20140413.2041)
 -->
<!-- Title: G Pages: 1 -->
 
 
 G 
 
<!-- 8864818972016196435 -->
 8864818972016196435 
 
 W 
 wishart_3 
 
<!-- 13032650889222424608 -->
 13032650889222424608 
 
 clamp_12 
 
<!-- 8864818972016196435--13032650889222424608 -->
 8864818972016196435--13032650889222424608 
 
 clamp_12 
 1 out 
 2 v 
 
<!-- 10690019871267403985 -->
 10690019871267403985 
 
 clamp_13 
 
<!-- 8864818972016196435--10690019871267403985 -->
 8864818972016196435--10690019871267403985 
 
 clamp_13 
 1 out 
 3 nu 
 
<!-- 3224467266032984958 -->
 3224467266032984958 
 
 clamp_1 
 
<!-- 182128913510163709 -->
 182128913510163709 
 
 clamp_9 
 
<!-- 11947672736640876909 -->
 11947672736640876909 
 
 clamp_15 
 
<!-- 17829735016521989593 -->
 17829735016521989593 
 
 clamp_4 
 
<!-- 7091879711336273879 -->
 7091879711336273879 
 
 ComplexNormal 
 complexnormal_1 
 
<!-- 7091879711336273879--11947672736640876909 -->
 7091879711336273879--11947672736640876909 
 
 clamp_15 
 1 out 
 4 C 
 
<!-- 12970611190394412270 -->
 12970611190394412270 
 
 HGF 
 hgf_1 
 
<!-- 7091879711336273879--12970611190394412270 -->
 7091879711336273879--12970611190394412270 
 
 X_1 
 1 X 
 2 μ 
 
<!-- 17265125386546587150 -->
 17265125386546587150 
 
 clamp_14 
 
<!-- 7091879711336273879--17265125386546587150 -->
 7091879711336273879--17265125386546587150 
 
 clamp_14 
 1 out 
 3 Γ 
 
<!-- 120400754430206087 -->
 120400754430206087 
 
 GM 
 gaussianmixture_1 
 
<!-- 12970611190394412270--120400754430206087 -->
 12970611190394412270--120400754430206087 
 
 ξ_1 
 1 out 
 2 ξ 
 
<!-- 4760280565994269124 -->
 4760280565994269124 
 
 Cat 
 categorical_1 
 
<!-- 3269805089122868117 -->
 3269805089122868117 
 
 Dir 
 dirichlet_1 
 
<!-- 4760280565994269124--3269805089122868117 -->
 4760280565994269124--3269805089122868117 
 
 _pi 
 1 out 
 2 p 
 
<!-- 10553431387828307260 -->
 10553431387828307260 
 
 clamp_7 
 
<!-- 10719337543648578384 -->
 10719337543648578384 
 
 W 
 wishart_1 
 
<!-- 10719337543648578384--17829735016521989593 -->
 10719337543648578384--17829735016521989593 
 
 clamp_4 
 1 out 
 2 v 
 
<!-- 10416481626776172838 -->
 10416481626776172838 
 
 clamp_5 
 
<!-- 10719337543648578384--10416481626776172838 -->
 10719337543648578384--10416481626776172838 
 
 clamp_5 
 1 out 
 3 nu 
 
<!-- 3269805089122868117--3224467266032984958 -->
 3269805089122868117--3224467266032984958 
 
 clamp_1 
 1 out 
 2 a 
 
<!-- 12388394097297614603 -->
 12388394097297614603 
 
 𝒩 
 gaussianmeanvariance_3 
 
<!-- 17676905737012181958 -->
 17676905737012181958 
 
 clamp_11 
 
<!-- 12388394097297614603--17676905737012181958 -->
 12388394097297614603--17676905737012181958 
 
 clamp_11 
 1 out 
 3 v 
 
<!-- 18306722582711782589 -->
 18306722582711782589 
 
 clamp_10 
 
<!-- 12388394097297614603--18306722582711782589 -->
 12388394097297614603--18306722582711782589 
 
 clamp_10 
 1 out 
 2 m 
 
<!-- 12935758529373277291 -->
 12935758529373277291 
 
 placeholder_y_1 
 
<!-- 12935758529373277291--7091879711336273879 -->
 12935758529373277291--7091879711336273879 
 
 y_1 
 1 out 
 1 out 
 
<!-- 13493657829259790451 -->
 13493657829259790451 
 
 𝒩 
 gaussianmeanvariance_1 
 
<!-- 3763211196341581102 -->
 3763211196341581102 
 
 clamp_3 
 
<!-- 13493657829259790451--3763211196341581102 -->
 13493657829259790451--3763211196341581102 
 
 clamp_3 
 1 out 
 3 v 
 
<!-- 12986133756278697260 -->
 12986133756278697260 
 
 clamp_2 
 
<!-- 13493657829259790451--12986133756278697260 -->
 13493657829259790451--12986133756278697260 
 
 clamp_2 
 1 out 
 2 m 
 
<!-- 7727640583

In [43]:
# Build the algorithm
q = PosteriorFactorization(_pi, m_1, w_1, m_2, w_2, m_3, w_3, z, X, ξ, ids=[:PI :M1 :W1 :M2 :W2 :M3 :W3 :Z :X :Ξ])
algo = variationalAlgorithm(q)

# Generate source code
source_code = algorithmSourceCode(algo);

# Load algorithm
eval(Meta.parse(source_code));

In [44]:
println(source_code)

begin

function stepZ!(data::Dict, marginals::Dict=Dict(), messages::Vector{Message}=Array{Message}(undef, 2))

messages[1] = ruleVBCategoricalOut(nothing, marginals[:_pi])
messages[2] = ruleVBGaussianMixtureZCat(marginals[:ξ_1], nothing, marginals[:m_1], marginals[:w_1], marginals[:m_2], marginals[:w_2], marginals[:m_3], marginals[:w_3])

marginals[:z_1] = messages[1].dist * messages[2].dist

return marginals

end

function stepΞ!(data::Dict, marginals::Dict=Dict(), messages::Vector{Message}=Array{Message}(undef, 2))

messages[1] = ruleVBGaussianMixtureOut(nothing, marginals[:z_1], marginals[:m_1], marginals[:w_1], marginals[:m_2], marginals[:w_2], marginals[:m_3], marginals[:w_3])
messages[2] = ruleVariationalHGFIn1PN(marginals[:X_1], nothing)

marginals[:ξ_1] = messages[1].dist * messages[2].dist

return marginals

end

function stepW3!(data::Dict, marginals::Dict=Dict(), messages::Vector{Message}=Array{Message}(undef, 2))

messages[1] = ruleVBWishartOut(nothing, ProbabilityDistribu

In [47]:
data = Dict(:y => [squeeze(Y)])

# Prepare posterior factors
marginals = Dict(:_pi => vague(ForneyLab.Dirichlet, N_clusters),
                 :m_1 => ProbabilityDistribution(Multivariate, GaussianMeanVariance, m=-1.0*ones(nr_freqs), v=1e4*diagm(ones(nr_freqs))),
                 :w_1 => vague(Wishart, nr_freqs),
                 :m_2 => ProbabilityDistribution(Multivariate, GaussianMeanVariance, m=ones(nr_freqs), v=1e4*diagm(ones(nr_freqs))),
                 :w_2 => vague(Wishart, nr_freqs),
                 :m_3 => ProbabilityDistribution(Multivariate, GaussianMeanVariance, m=ones(nr_freqs), v=1e4*diagm(ones(nr_freqs))),
                 :w_3 => vague(Wishart, nr_freqs))
for i in 1:n_samples
    marginals[:z_*i] = vague(Categorical)
    marginals[:X_*i] = ProbabilityDistribution(Multivariate, ComplexNormal, μ=zeros(nr_freqs) .+ 0.0im, Γ=1e-10*diagm(ones(nr_freqs)).+0im, C=mat(0.0+0.0im));
    marginals[:ξ_*i] = ProbabilityDistribution(ForneyLab.Multivariate, GaussianMeanVariance, m=zeros(nr_freqs), v=diagm(ones(nr_freqs)))
end



In [48]:
# Execute algorithm
n_its = 10
@showprogress for i in 1:n_its
    stepX!(data, marginals)
    stepΞ!(data, marginals)
    stepZ!(data, marginals)
    stepPI!(data, marginals)
    stepM1!(data, marginals)
    stepW1!(data, marginals)
    stepM2!(data, marginals)
    stepW2!(data, marginals)   
end

Progress: 100%|█████████████████████████████████████████| Time: 0:00:06
